In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/'
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"
data_time = '20180920'

In [3]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [4]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [5]:
ENWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA):
    if file.endswith("art.csv.gz"):
        ENWIKI_ART_FNMS.append(file)
ENWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA):
    if file.endswith("red.csv.gz"):
        ENWIKI_RED_FNMS.append(file)   

In [6]:
print('ENWIKI_ART_FNMS:', len(ENWIKI_ART_FNMS))
print('ENWIKI_RED_FNMS:', len(ENWIKI_RED_FNMS))

ENWIKI_ART_FNMS: 55
ENWIKI_RED_FNMS: 55


In [7]:
df_en_id_name = None
for fn in ENWIKI_ART_FNMS:
    fn = PATH_TO_DATA+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    print(df_articles.head(100))
    df_id_name_tmp = df_articles[['id', 'title', 'text_len']].drop_duplicates()
    df_id_name_tmp.columns = ['id', 'title', 'length']
    #print("df_id_name_tmp size: {}".format(df_id_name_tmp.shape))
    if df_en_id_name is not None:
        df_en_id_name = df_en_id_name.append(df_id_name_tmp)        
        #print("append")
    else:
        df_en_id_name = df_id_name_tmp
        #print("assign")
    print("df_en_id_name size: {}".format(df_en_id_name.shape))
    os.remove(fn_new)
df_en_id_name = df_en_id_name.drop_duplicates()

/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current14.xml-p6197598p7697598.bz2_art.csv.gz
         id                 title  text_len  link_pos  \
0   6197600  Flying Tigers (film)     17803       134   
1   6197600  Flying Tigers (film)     17803       245   
2   6197600  Flying Tigers (film)     17803       263   
3   6197600  Flying Tigers (film)     17803       304   
4   6197600  Flying Tigers (film)     17803       336   
5   6197600  Flying Tigers (film)     17803       396   
6   6197600  Flying Tigers (film)     17803       542   
7   6197600  Flying Tigers (film)     17803       724   
8   6197600  Flying Tigers (film)     17803       757   
9   6197600  Flying Tigers (film)     17803      1106   
10  6197600  Flying Tigers (film)     17803      1140   
11  6197600  Flying Tigers (film)     17803      1175   
12  6197600  Flying Tigers (film)     17803      1208   
13  6197600  Flying Tiger

KeyboardInterrupt: 

In [9]:
df_en_id_name = df_en_id_name[['id', 'title']]

In [10]:
df_en_id_name.to_csv(PATH_TO_DATA+"enwiki-" + data_time + "-id_name.csv", 
                     index = False, encoding='UTF-8', quotechar="\"", sep ="^")

In [8]:
df_en_id_name = pd.read_csv(PATH_TO_DATA+"enwiki-" + data_time + "-id_name.csv", encoding='utf-8', sep="^")
df_en_id_name.head()

,id,title
0,6197600,Flying Tigers (film)
1,6197604,Mariano Andreu
2,6197607,David Prain
3,6197617,Rut Brandt
4,6197621,Gregory C. Coleman


In [9]:
df_en_id_name.shape

(5719743, 2)

In [ ]:
# go to alias grabber file

In [10]:
df_en_id_orig_title_alias = pd.read_csv(PATH_TO_DATA+"enwiki-" + data_time + "-id_orig_title_alias.csv",  encoding='UTF-8', quotechar="\"")
df_en_id_orig_title_alias.head()

,id_orig,title_alias
0,954897,Shaheedka
1,56160400,Barbarossa cycling trail
2,36339503,Jessica Lange on screen and stage
3,56160400,Barbarossa cycleway
4,20239787,Tomás O'Ryan


In [11]:
df_en_id_orig_title_alias.columns = ['id', 'title']
df_en_id_name_all = df_en_id_name.append(df_en_id_orig_title_alias)
df_en_id_name_all.shape

(14127867, 2)

In [19]:
#ignored redirections
#df_en_id_name_all = df_en_id_name

In [12]:
df_en_id_name_all['title'] = df_en_id_name_all['title'].str.strip()

In [13]:
df_en_id_name_all['title_lc'] = df_en_id_name_all['title'].str.lower()
df_en_id_name_all = df_en_id_name_all[['id', 'title_lc']]
df_en_id_name_all = df_en_id_name_all.drop_duplicates()
df_en_id_name_all.shape

(13369526, 2)

In [14]:
df_en_id_name_all.head()

,id,title_lc
0,6197600,flying tigers (film)
1,6197604,mariano andreu
2,6197607,david prain
3,6197617,rut brandt
4,6197621,gregory c. coleman


In [21]:
for fn in ENWIKI_ART_FNMS:
    fn = PATH_TO_DATA+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='ISO-8859-1', quotechar="\"")
    df_articles['link_val'] = df_articles['link_val'].str.split("#").str[0]
    #df_articles = df_articles[df_articles['link_val'].notna()]
    df_articles = df_articles[df_articles['link_val'].str.strip() != '']
    df_articles['link_pos_perc'] = (df_articles['link_pos']/df_articles['text_len']).round(3)       
    df_articles['link_val_lc'] = df_articles['link_val'].str.lower()
    df_en_titles_series = df_articles.iloc[:,7]
    df_en_titles_series_stripped = df_en_titles_series.str.replace('  ', ' ')
    df_titles_stripped = df_en_titles_series_stripped.to_frame()
    df_articles_less = df_articles[['id', 'title', 'link_val']]
    df_articles = pd.concat([df_articles_less, df_titles_stripped], axis=1)
    df_articles['link_val_lc'] = df_articles['link_val_lc'].str.strip()
    df_articles = pd.merge(df_articles, df_en_id_name_all, how='left', left_on=['link_val_lc'], right_on=['title_lc'])
    df_articles = df_articles[['id_x', 'id_y', 'link_val']]
    df_articles.columns = ['id', 'link_id', 'link_val']
    df_articles = df_articles[pd.notnull(df_articles['link_val'])]
    df_articles[['link_id']] = df_articles[['link_id']].fillna(-1).astype(int)
    df_articles['is_red_link'] = np.where(df_articles['link_id'] == -1, True, False)
    df_articles.to_csv(fn_new, index = False, encoding='ISO-8859-1', quotechar="\"")
    pack_and_remove(fn_new)

/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current14.xml-p6197598p7697598.bz2_art.csv.gz
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current27.xml-p54663462p56163462.bz2_art.csv.gz
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current27.xml-p53163462p54663462.bz2_art.csv.gz
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current15.xml-p7744803p9244803.bz2_art.csv.gz
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current25.xml-p35452816p36952816.bz2_art.csv.gz
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwik

In [41]:
!pwd

/media/andrii/earth/Katia/BabelNet_baseline/uk-en_redlinks/code/get_redlinks_from_dumps


In [19]:
df_articles = pd.read_csv(PATH_TO_DATA+'enwiki-20180920-pages-links.csv', encoding='UTF-8', quotechar="\"")

In [20]:
df_articles

,id,link_id,link_val,is_red_link
0,28323668,-1,UniversitÃ© de MontrÃ©al,True
1,28323668,-1,UniversitÃ© de MontrÃ©al,True
2,28323691,-1,Primera DivisiÃ³n Peruana,True
3,28323691,1232972,Alianza Lima,False
4,28323691,-1,Jorge ChÃ¡vez Nr. 1,True
5,28323691,32089546,1917 Liga Peruana de Football,False
6,28323691,28323797,1919 Liga Peruana de Football,False
7,28323691,-1,Peruvian Primera DivisiÃ³n,True
8,28323691,1232972,Alianza Lima,False
9,28323691,1232972,Alianza Lima,False


In [6]:
df_id_name = pd.read_csv(PATH_TO_DATA+'enwiki-' + data_time + '-id_name.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")

In [21]:
df_id_name.head()

,id,title
0,12,Anarchism
1,25,Autism
2,39,Albedo
3,290,A
4,303,Alabama


In [18]:
fn = PATH_TO_DATA+"enwiki-" + data_time + "-langlinks_en.csv"
df_en_langlinks = pd.read_csv(fn, encoding='UTF-16', sep="^")

In [27]:
df_en_langlinks = df_en_langlinks[pd.notnull(df_en_langlinks['ll_title'])]

In [28]:
df_en_langlinks.head()

,ll_from,ll_lang,ll_title
23,2112530,en,! (The Dismemberment Plan album)
24,1366003,en,! (disambiguation)
25,2141483,en,!!
26,425480,en,!!!
27,848226,en,!Action Pact!


In [23]:
#df_en_en_links = pd.merge(df_en_langlinks, df_id_name, how='left', left_on=['ll_title'], right_on=['title'])

In [ ]:
df_en_en_links = df_en_en_links[['ll_from', 'id']]
df_en_en_links.columns = ['id_en', 'id_en']
df_en_en_links[['id_en']] = df_en_en_links[['id_en']].fillna(-1).astype(int)
df_en_en_links[['id_en']] = df_en_en_links[['id_en']].fillna(-1).astype(int)
df_en_en_links.to_csv(PATH_TO_DATA_en + data_time + "-langlinks_en_en.csv", index = False, 
                      encoding='ISO-8859-1', quotechar="'", 
                      escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)
pack_and_remove(fn_new)

In [30]:
df_en_langlinks.to_csv(PATH_TO_DATA + data_time + "-langlinks_en_en.csv", index = False, 
                      encoding='ISO-8859-1', quotechar="'", 
                      escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)